In [3]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt

# Мерджнутые таблицы

In [ ]:
df_merged = pd.read_csv("merged_left.csv").drop(columns=['Unnamed: 0', 'page'])
df_merged

In [ ]:
df_merged.info()

# Поработаем с данными и произведем первый этап чистки

## Начнем с анализа Title и name_parsed
из info видно:

*   non-null для title- 12158
*   non-null для title- 13960

In [ ]:
mask = df_merged["title"].isna() & df_merged["name_parsed"].notna()
df_merged_title = df_merged[mask]
df_merged_title

In [7]:
print(f'количесвто таких случаев: {df_merged_title.shape[0]}')

количесвто таких случаев: 1802


In [ ]:
df_merged_title.info()

In [ ]:
def extract_game_name(url):
    match = re.search(r"/game/([^/]+)/?", str(url))
    return match.group(1) if match else None


df_merged_title["url_name"] = df_merged_title["url"].apply(extract_game_name)
df_merged_title_1=df_merged_title[['title','name_parsed','url_name']]
df_merged_title_1

In [ ]:
df_merged_title_1["url_name_clean"] = df_merged_title_1["url_name"].str.replace("-", "", regex=False)

df_merged_title_1.drop(columns=['url_name'])

In [ ]:
def clean_name(name):
    name = name.lower()
    name = re.sub(r"[^a-z0-9]", "", name)
    return name
df_merged_title_1["name_clean"] = df_merged_title_1["name_parsed"].apply(clean_name)
df_merged_title_1= df_merged_title_1.drop(columns=['url_name', 'name_parsed','title'])

In [12]:
matches = df_merged_title_1["url_name_clean"] == df_merged_title_1["name_clean"]
count_matches = matches.sum()
total_rows = len(df_merged_title_1)
print(f"Совпадений: {count_matches} из {total_rows}")
print(f"Доля совпадений: {count_matches / total_rows:.2%}")

Совпадений: 1713 из 1802
Доля совпадений: 95.06%


In [ ]:
mismatched = df_merged_title_1[df_merged_title_1["url_name_clean"] != df_merged_title_1["name_clean"]]
mismatched[["url_name_clean", "name_clean"]]

все кто не совпал это либо подстроки  либо же одинаковая суть названия но они немного разные и это нормально


*   madagascar3thevideogame - dreamworksmadagascar3thevideogame
*   gardeningmamamamatomorinonakamatachi - gardeningmama2forestfriends



In [ ]:

mismatched["substring_match"] = mismatched.apply(
    lambda row: (row["url_name_clean"] in row["name_clean"]) or (row["name_clean"] in row["url_name_clean"]),
    axis=1
)
print("Всего несовпадений:", len(mismatched))
print("Из них содержат подстроку друг друга:", mismatched["substring_match"].sum())
mismatched[mismatched['substring_match']==True]


40 остальных это та же самая игра и значит можно просто дозаполнить пропуски в начальной таблице  в строке title

Сделаем один столбец с названиями

In [15]:
df_merged["title"] = df_merged["title"].fillna(df_merged["name_parsed"])

filled_count = df_merged["title"].isna().sum()
print(f"Осталось пустых title: {filled_count}")

Осталось пустых title: 0


## Продолжим анализом metascore и user_score

Рассмотрим именно их потому что возник тип данных object и это интересно

In [ ]:
mask = pd.to_numeric(df_merged["metascore"], errors="coerce").isna()
non_numeric = df_merged[mask]

print(f"Всего нечисловых значений: {len(non_numeric)}")
non_numeric[["metascore"]]

In [17]:
df_merged["metascore"] = pd.to_numeric(df_merged["metascore"], errors="coerce")

In [ ]:
mask = pd.to_numeric(df_merged["user_score"], errors="coerce").isna()
non_numeric = df_merged[mask]

print(f"Всего нечисловых значений: {len(non_numeric)}")
non_numeric[["user_score"]]

In [19]:
df_merged["user_score"] = pd.to_numeric(df_merged["user_score"], errors="coerce")

In [20]:
df_merged=df_merged.drop(columns=['name_parsed', 'metacritic', 'released','rating_api','url', 'summary'])

In [21]:
def clean_rating(rating):
    if pd.isna(rating):
      return np.nan
    else:
      rating_str = str(rating)
      return re.sub(r'^Rated\s+', '', rating_str)
    return rating_str
df_merged['rating_parsed'] = df_merged['rating_parsed'].apply(clean_rating)

In [ ]:
df_merged.to_csv("merged_clean(last_version).csv", index=False, sep=";")
df_merged